In [ ]:
dbutils.fs.unmount('/mnt/sales')

In [1]:
dbutils.fs.mount(
source = 'wasbs://sales@retaildbsales.blob.core.windows.net',
mount_point = '/mnt/sales',
extra_configs={'fs.azure.account.key.retaildbsales.blob.core.windows.net':'eosasFhF9qyCVCwLdVvWoIK0kbL7Zj1ppaQmkjOeVDPfGTZ/Otp1NYHoKGicNTpmmuWldhm63M1L+AStZAKZjg=='})

NameError: name 'dbutils' is not defined

In [ ]:
%fs 
ls /mnt/sales

path,name,size,modificationTime
dbfs:/mnt/sales/discarded/,discarded/,0,0
dbfs:/mnt/sales/landing/,landing/,0,0
dbfs:/mnt/sales/staging/,staging/,0,0


In [ ]:
%fs 
ls /mnt/sales/landing


path,name,size,modificationTime
dbfs:/mnt/sales/landing/orders.csv,orders.csv,224,1724833484000


In [ ]:
df = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("/mnt/sales/landing/orders.csv")

In [ ]:
df.show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
| 1111111|2013-07-25 00:00:00|      11599|         CLOSED|
| 2222222|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
| 3333333|2013-07-25 00:00:00|      12111|       COMPLETE|
| 4444444|2013-07-25 00:00:00|       8827|         CLOSED|
+--------+-------------------+-----------+---------------+



In [ ]:
display(df)

order_id,order_date,customer_id,order_status
1111111,2013-07-25T00:00:00Z,11599,CLOSED
2222222,2013-07-25T00:00:00Z,256,PENDING_PAYMENT
3333333,2013-07-25T00:00:00Z,12111,COMPLETE
4444444,2013-07-25T00:00:00Z,8827,CLOSED


In [ ]:
errorFlg = False

orderscount = df.count()
distinct_orders = df.select("order_id").distinct().count()

if orderscount != distinct_orders:
    errorFlg = True
if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/orders.csv', 'mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg": "true", "errorMsg":"Orderid is repeated"}')
    
df.createOrReplaceTempView("orders")

In [ ]:
print(distinct_orders)
print(orderscount)

4
4


In [ ]:
dbServer = 'retaidb-sales'
dbPort = '1433'
dbName = 'retaildb'
dbUser = 'prem'
dbPassword = 'sqlpassword'
databricksScope = 'salesprojectscope'

connectionUrl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer,dbPort, dbName, dbUser)

dbPassword = dbutils.secrets.get(scope = databricksScope, key='sqlpassword')

connectionProperties = {'password': dbPassword,'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'}

In [ ]:
valid_status_df = spark.read.jdbc(url = connectionUrl, table = 'dbo.valid_order_status', properties = connectionProperties)

In [ ]:
valid_status_df.show()

+---------------+
|    status_name|
+---------------+
|        ON_HOLD|
| PAYMENT_REVIEW|
|     PROCESSING|
|         CLOSED|
|SUSPECTED_FRAUD|
|       COMPLETE|
|        PENDING|
|       CANCELED|
|PENDING_PAYMENT|
+---------------+



In [ ]:
valid_status_df.createOrReplaceTempView('valid_status')

In [ ]:
invalid_rows_df = spark.sql("select * from orders where order_status not in (select * from valid_status)")

In [ ]:
invalid_rows_df.show()

+--------+----------+-----------+------------+
|order_id|order_date|customer_id|order_status|
+--------+----------+-----------+------------+
+--------+----------+-----------+------------+



In [ ]:
if invalid_rows_df.count() > 0:
    errorFlg = True
if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/orders.csv', 'mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg": "true", "errorMsg":"invalid Order status found}')
else:
    dbutils.fs.mv('/mnt/sales/landing/orders.csv', 'mnt/sales/staging')
    dbutils.notebook.exit('{"errorFlg": "false", "errorMsg":"all good"}')
    